# ALOS-2 to RSLC
- This notebook converts ALOS data to NISAR L0B and then RSLC data by running ISCE3's `alos2_to_nisar_l1.py`.
- Uses the `isce3_src` kernel (created using [Create_Environments.ipynb](https://github.com/isce-framework/sds-ondemand/blob/main/environments/Create_Environments.ipynb)).
- Can be ran locally (preferably on a GPU instance), or as a PCM job.

# Parameters
This cell is marked `parameters`, indicating the variables within can substituted when running this notebook via `papermill`.
- `data_link`: S3 url to the ALOS-1 data to be converted into NISAR RSLC.
- `gpu_enabled`: `1` to run using the GPU, `0` to use CPU instead. **Keep in mind that while disabling the GPU processing allows this notebook to be ran on an instance without a GPU, that does not guarantee the instance this notebook is running on is a non-GPU instance.** To run on a non-GPU instance on PCM, submit the job to a CPU-only queue.

### Upload parameters (PCM only)
- `timestamp`: A time-string of the format `%Y%m%dT%H%M%S` indicating the time at which the job was submitted. This helps `pcm.py` find where the results of this job will be submitted when this notebook is ran as a PCM job.

In [ ]:
data_link = 's3://nisar-st-data-ondemand/alex/ALOS2_zip/0000511819_001001_ALOS2482920840-230503.zip' # string
gpu_enabled = 1 # boolean
timestamp = '20240131T0123456' # string

# hysds specifications
_time_limit = 86400
_soft_time_limit = 86400
_disk_usage = '10GB'
_submission_type = 'iteration'
_label = 'ALOS-2 to RSLC PGE'

### Pre-processing of the Parameters to convert numbers or words into `boolean` True and False values.

In [ ]:
# Convert boolean parameters because they must be specified as strings
try:
    if not isinstance(gpu_enabled, bool):
        gpu_enabled = int(gpu_enabled) > 0
except ValueError:
    if instance(gpu_enabled, str):
        gpu_enabled = gpu_enabled.lower() == 'true'
    else:
        gpu_enabled = False
print(f'{gpu_enabled=}')

# Extraneous parameters
insar_config = '' # string

# Functions for loading runconfig files and downloading from S3 buckets

In [ ]:
import os
import yaml
import asf_search as asf
import boto3
import aws_uploader

WORKING_DIR = os.getcwd()
HOME_DIR = os.environ['HOME']
NOTEBOOK_PGE_DIR = os.environ.get('NOTEBOOK_PGE_DIR', WORKING_DIR)
ISCE3_BUILD_DIR = os.environ.get('ISCE3_BUILD_DIR', f'{HOME_DIR}/isce3/build')

DOWNLOAD_DIR = os.path.join(WORKING_DIR, 'downloads')
EXTRACT_DIR = os.path.join(WORKING_DIR, 'alos_data')
OUTPUT_DIR = os.path.join(WORKING_DIR, 'output')
PRODUCT_DIR = os.path.join(WORKING_DIR, 'product_path')

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(WORKING_DIR)

# Run ISCE3 Python Scripts
This cell runs the python scripts:
- `alos2_to_nisar_l1.py`: Converts ALOS-2 data -> NISAR RSLC

In [ ]:
from urllib.parse import urlparse
import isce3_config

# Download the specified ALOS data
extract_dir = isce3_config.download_alos2_s3_data(data_link, DOWNLOAD_DIR, EXTRACT_DIR)
output_f = os.path.join(PRODUCT_DIR, f'{os.path.basename(extract_dir)}.h5')
print(f'ALOS-2 to NISAR L1 Destination: {output_f}')
print(f'Executing:\n    mamba run -n isce3_src python {ISCE3_BUILD_DIR}/share/nisar/examples/alos2_to_nisar_l1.py -i {extract_dir} -o {output_f}')
!mamba run -n isce3_src python {ISCE3_BUILD_DIR}/share/nisar/examples/alos2_to_nisar_l1.py -i {extract_dir} -o {output_f}

# Automatic Stage Out for PCM Jobs
This routine creates a folder which matches the regex:
```
(?P<id>NISAR_L1_(?P<processing_type>[P|U]R)_RSLC_(?P<cycle_number>\d{3})_(?P<relative_orbit_number>\d{3})_(?P<Direction>\w)_(?P<TrackFrame>\d{3})_(?P<radar_processing_mode>\d{4})_(?P<Polarization>\w{2})_(?P<RadarStartDateTime>(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2})T\d{6})_(?P<RadarStopDateTime>\d{8}T\d{6})_(?P<composite_release_id>\w{6})_(?P<Fidelity>\w)_(?P<CoverageIndicator>[F|P])_(?P<product_counter>\d{3}))$
```
This enables PCM to automatically upload the results of this notebook to the `s3://nisar-{system_name}-rs-ondemand` bucket.

`system_name` is one of the following:
- `st` (for Science Team)
- `adt` (for ADT)
- `iot` (for IOT)

In [ ]:
import h5py
import json

# Grab some appropriate values for PCM to automatically stage out
# try:
#     h5_file = h5py.File(output_f, 'r')
#     polarization = h5_file['science']['LSAR']['RSLC']['swaths']['frequencyA']['listOfPolarizations'][0].decode('utf-8')
# except Exception as e:
#     print(f'Exception occured: {e}')
#     print(f'As the polarization was unable to be automatically detected, the polarization will be listed as \'HH\' by default.')
polarization = 'HH'
regex_name = isce3_config.RSLC_FORMAT.format(
    polarization=polarization,
    timestamp=timestamp)
auto_output_dir = os.path.join(WORKING_DIR, regex_name)

# Create the automatic stage-out directory
!mkdir -p {auto_output_dir}
!cp {WORKING_DIR}/*-output.ipynb {auto_output_dir}
!cp {WORKING_DIR}/*.txt {auto_output_dir}
!cp {WORKING_DIR}/*.json {auto_output_dir}
!mv {output_f} {auto_output_dir}

# Move the new path variables to avoid confusion
output_f = os.path.join(auto_output_dir, os.path.basename(output_f))

# Create metadata files for the automatic stage-out directory
with open(os.path.join(auto_output_dir, f'{regex_name}.met.json'), 'w', encoding='utf-8') as f:
    content = {
        'polarization': polarization,
    }
    f.write(json.dumps(content))
with open(os.path.join(auto_output_dir, f'{regex_name}.dataset.json'), 'w', encoding='utf-8') as f:
    content = {
         'version': 'v1.0',
         'label': 'This is purely an EXAMPLE metadata file, the values in this file are not representative of this product.',
         'location': {
           'type': 'polygon',
           'coordinates': [
             [
                [-122.9059682940358,40.47090915967475],
                [-121.6679748715316,37.84406528996276],
                [-120.7310161872557,38.28728069813177],
                [-121.7043611684245,39.94137004454238],
                [-121.9536916840953,40.67097860759095],
                [-122.3100379696548,40.7267890636145],
                [-122.7640648263371,40.5457010812299],
                [-122.9059682940358,40.47090915967475]
              ]
            ]
        },
        'starttime': '2017-01-01T00:00:00',
        'endtime': '2017-01-01T00:05:00',
    }
    f.write(json.dumps(content))